In [11]:
# Install package for easyocr
!pip install easyocr

In [9]:
# Install streamlit package and menu options
!pip install streamlit streamlit_option_menu

In [3]:
# Install local tunnel
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [5]:
# Generate application file in python

%%writefile app1.py
import easyocr
import numpy as np
import re
import pandas as pd
import sqlite3
import streamlit as st
from streamlit_option_menu import option_menu
from PIL import Image

# Creating Streamlit page with sidebar & menu
st.title('EXTRACTING BUSINESS CARD DATA WITH OCR')

with st.sidebar:
    select = option_menu('Main Menu', ['Home', 'Upload & Extract', 'Delete'])

try:
  # Select Delete option
  if select =="Delete":
    mycon = sqlite3.connect('bizcardx.db')
    mycursor = mycon.cursor()
    query = "SELECT Name, Designation, Email, Phone, Area, State, Pincode FROM Card_Details"
    df = pd.read_sql_query(query, mycon) # retrieves data-table from SQLite
    st.dataframe(df)

    # Select a row to delete
    row_to_delete = st.selectbox("Select the row to delete:", df.index)

    # Button to delete the selected row
    if st.button("Delete Row"):
        df = df.drop(row_to_delete)
        df.to_sql('Card_Details', mycon, if_exists='replace', index=False)
        st.write("Updated DataFrame:")
        st.dataframe(df[['Name', 'Designation', 'Email', 'Phone', 'Area', 'State', 'Pincode']])

except:
  st.text("No Data found!!")
finally:
  # Select Image upload & extract option
  if select =="Upload & Extract":
    uploaded_image = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    # Check if a file has been uploaded
    if uploaded_image is not None:

      # Display image file
      image_data = uploaded_image.read()
      image = Image.open(uploaded_image)
      st.image(image, caption='Uploaded Image', use_container_width=True)

      # Convert image to a format suitable for EasyOCR
      image_np = np.array(image)
      reader = easyocr.Reader(['en'], gpu=False)
      result = reader.readtext(image_np)
      Detected_text = []

      # Convert detected text to string format
      for (bbox, text, prob) in result:
        Detected_text.append(text)
        Detected_text_str = " ".join(Detected_text)

      # Finding required string using regular expressions
      name = Detected_text[0]
      designation = Detected_text[1]
      email = re.findall('\S+@\S+', Detected_text_str)
      phone = re.findall('\+?[\d]{2,}\-[\d]{3}\-[\d]{4}', Detected_text_str)
      area = re.findall('[\d]{1,3}\s[a-zA-Z]{2,7}\s[a-zA-Z]{2,}',Detected_text_str)
      try:
        if area[1]:
          area = area[1]
      except:
          area = area[0]
      state = re.findall('[\S]{5,}\,?\;?\s[\d]{6,}',Detected_text_str)
      state = re.split('\s|\;|\,',state[0])
      pincode = re.findall('[\d]{6,7}',Detected_text_str)

      # Display detected card details
      card_data = {
                      'Name': name,
                      'Designation':designation,
                      'Email':email[0],
                      'Phone':phone[0],
                      'Area':area,
                      'State':state[0],
                      'PinCode':pincode[0]
      }
      df = pd.DataFrame([card_data])
      st.dataframe(df)

      # Save card details to SQLite
      if st.button('Save Data'):
        mycon = sqlite3.connect('bizcardx.db')   # change the name of database accordingly
        mycursor = mycon.cursor()  # create a cursor
        query1 = "create table if not exists Card_Details (Name varchar(100), Designation varchar(100), Email varchar(50), Phone int, Area varchar(100), State varchar(50), Pincode int, Image BLOB)"
        mycursor.execute(query1)

        try:
          query2 = "insert into Card_Details (Name, Designation, Email, Phone, Area, State, Pincode, Image) values (?,?,?,?,?,?,?,?)"
          val = (name, designation, email[0], phone[0], area, state[0], pincode[0], image_data)
          mycursor.execute(query2,val)
        except:
          query3 = "ALTER TABLE Card_Details ADD COLUMN Image BLOB"
          mycursor.execute(query3)
          mycursor.execute(query2,val)

        mycon.commit()
        st.write('Card details saved successfully!!')


Writing app1.py


In [12]:
!wget -q -O - ipv4.icanhazip.com

35.193.115.141


In [13]:
! streamlit run app1.py & npx localtunnel --port 8501

⠙

your url is: https://ripe-deer-serve.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.193.115.141:8501

2025-03-31 07:13:36.938 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_module_paths
    potential_paths = extract_paths(module)
                      ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 210, in <lambda>
    lambda m: list(m.__path__._